In [ ]:
%load frap_analysis.py

# Example FRAP analysis

### Read FRAP data

In [ ]:
# get all data
path = "example_frap_output.csv" # path to output from FRAP ROI correction
all_datasets = []
all_names = []
condition_names = []
for condition in listdir_nohidden(path):
    if condition != "excluded":
        print(condition)
        dataset, names = read_all_movies(os.path.join(path, condition))
        all_datasets.append(dataset)
        all_names.append(names)
        condition_names.append(condition)
all_datasets_no_exclusion = [dataset for dataset, name in zip(all_datasets, condition_names) if name != "exclusion"]
condition_names_no_exclusion = [name for name in condition_names if name != "exclusion"]

# reorder based on final values
mean_final_values = []
for datasets in all_datasets:
    combined_data = pd.concat(datasets)
    combined_data = combined_data[["time", "intensity", "label"]].reset_index()
    combined_final_values = combined_data[combined_data["time"] > 500]
    mean_final_values.append(np.mean(combined_data["intensity"].to_numpy()))
    
zipped = zip(mean_final_values, all_datasets, all_names, condition_names)
mean_final_values_ordered, all_datasets_ordered, all_names_ordered, condition_names_ordered = zip(*sorted(zipped))

### Examine all of the tracks

In [ ]:
i = 1 # provide index for desired condition
print(condition_names_ordered[i])
plot_all_plots_square(all_datasets_ordered[i], all_names_ordered[i])

### Examine average fit for one condition

In [ ]:
plot_data_model(all_datasets_ordered[i], weight_fit=False, enforce_bleach_frame=True, bleach_idx=10)

### Plot multiple model fits

In [ ]:
# make plots
print(condition_names_ordered)
condition_names_no_dep = tuple([condition_names_ordered[0]]) + condition_names_ordered[2:4] + \
                        tuple([condition_names_ordered[5]])
print(condition_names_no_dep)
all_datasets_no_dep = tuple([all_datasets_ordered[0]]) + all_datasets_ordered[2:4] + \
                        tuple([all_datasets_ordered[5]])
summary_data, _ = plot_multiple_datasets(all_datasets_no_dep, condition_names_no_dep, 
                                         plot_model=True, weight_fit=True, 
                                         enforce_bleach_frame=True, bleach_idx=10)
sns.despine()
plt.savefig("frap_output.svg", format="svg")